In [1]:
import pandas as pd
import seaborn as sns

from utils.tags import tag_elements

__Загрузка датафрейма__

Данные можно получить не из локального файла, а по ссылке:

https://drive.google.com/uc?id=1_osI1ZMtR3chrKtwRfwlNyrr5VP0OIT0

In [2]:
article_data = pd.read_csv('./datasets/article_about_data.gz',
                           compression='gzip',
                           parse_dates=['datetime'],
                           index_col='id')

__Предобработка данных__

_Предобработка тэгов_

In [3]:
# Теги загружены в виде строки вида ['tag_1', 'tag_1', 'tag_1'],
# привожу его к массиву.

article_data.tags = [tag for tag in article_data.tags.str.replace(
                     pat="[\[\]']", repl='', regex=True).str.split(',') if tag != '']

# Раскрываю датафрейм по колонке tags. Применяю к каждому отдельному тегу tag_elements.

article_data = article_data.explode('tags')
article_data.tags = article_data.tags.apply(tag_elements)

In [ ]:
# Выбираю тэги и подсчитываю их встречаемость.

tags = article_data.groupby('tags', as_index=False) \
                   .agg({'comments': 'count'}) \
                   .rename(columns={'tags': 'tag', 'comments': 'used'}) \
                   .sort_values(['used', 'tag'])

min_tag_used = 100
max_tag_used = 10_000

# Удаляю редко используемые "мусорные" тэги
tags.drop(tags.query('used <= @min_tag_used').index, inplace=True)
tags.drop(tags.query('used >= @max_tag_used').index, inplace=True)

# Удаляю пустые тэги
tags.drop(tags.query('tag == ""').index, inplace=True)

# Определяю язык тэга и удаляю теги на двух языках
tags.drop(tags[(tags.tag.str.contains('^[а-я]+$', regex=True) == False) & (tags.tag.str.contains('^[a-z]+$', regex=True) == False)].index, inplace=True)

In [ ]:
# Совмещаю данные по тэгам с исходным датафреймом.

article_data = article_data.merge(tags, left_on='tags', right_on='tag', how='left') \
                           .drop(columns='tags') \
                           .dropna()

__Это на потом, может быть использую.__

In [ ]:
# 'https://drive.google.com/uc?id=1YjYmulVuJPH0IP770enNPWo9WPHdf1U0'

# article_text = pd.read_csv('./datasets/article_text_data.gz',
#                            compression='gzip',
#                            index_col='id')
